## Guidance Basics

- [GitHub Repo](https://github.com/guidance-ai/guidance/tree/main)
- [Docs](https://guidance.readthedocs.io/en/latest/)


In [ ]:
import guidance

In [ ]:
# this relies on the environment variable OPENAI_API_KEY being set
llm = guidance.llms.OpenAI('text-davinci-003')

program = guidance("""My favorite flavor is{{gen 'flavor' max_tokens=10 stop="."}}""", llm=llm)
program()